### Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №3 - Дерево решений


**Общая информация**

**Срок сдачи:** до 30 апреля 2018, 06:00   
**Штраф за опоздание:** -2 балла после 06:00 30 апреля, -4 балла после 06:00 7 мая, -6 баллов после 06:00 14 мая, -8 баллов после 06:00 21 мая

При отправлении ДЗ указывайте фамилию в названии файла   


Присылать ДЗ необходимо в виде ссылки на свой github репозиторий в slack @alkhamush


Используйте данный Ipython Notebook при оформлении домашнего задания.

###### Задание 1 (3 балла)
Разберитесь в коде MyDecisionTreeClassifier, который уже частично реализован. Допишите код там, где написано "Ваш код". Ваша реализация дерева должна работать по точности не хуже DecisionTreeClassifier из sklearn. Точность проверяется на wine и Speed Dating Data.

###### Задание 2 (3 балла)
Добиться скорости работы на fit не медленнее чем в 10 раз sklearn на данных wine и Speed Dating Data. 
Для этого используем numpy.

###### Задание 3 (2 балла)
Добавьте функционал, который определяет значения feature importance. Выведите 10 главных фичей под пунктом Задание 4 (уже написано ниже) для MyDecisionTreeClassifier и DecisionTreeClassifier так, чтобы сразу были видны выводы и по MyDecisionTreeClassifier, и по DecisionTreeClassifier. Используем данные Speed Dating Data.

###### Задание 4 (2 балла)
С помощью GridSearchCV или RandomSearchCV подберите наиболее оптимальные параметры для случайного леса (Выберете 2-3 параметра). Используем данные Speed Dating Data. Задание реализуйте под пунктом Задание 5 (уже написано ниже)


**Штрафные баллы:**

1. Невыполнение PEP8 -1 балл
2. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw3.ipynb) -1 балл
3. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -1 балл
4. При оформлении ДЗ нужно пользоваться данным файлом в качестве шаблона. Не нужно удалять и видоизменять написанный код и текст. В противном случае -1 балл

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline
%load_ext pycodestyle_magic

In [2]:
#%%pycodestyle


class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=None,
                 sufficient_share=1.0, criterion='gini', max_features=None):
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        self.feature_importances_ = None
        if criterion == 'gini':
            self.G_function = self.__gini
            self.G_impurity = self.__gini_i
        elif criterion == 'entropy':
            self.G_function = self.__entropy
            self.G_impurity = self.__entropy_i
        elif criterion == 'misclass':
            self.G_function = self.__misclass
            self.G_impurity = self.__misclass_i
        else:
            print('invalid criterion name')
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features is None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print('invalid max_features name')
            raise

    def __gini_i(self, y):
        y = y.astype('float')
        return 1 - np.dot(y, y.T) / np.sum(y)**2

    def __misclass_i(self, y):
        y = y.astype('float')
        return 1 - np.max(y) / np.sum(y)

    def __entropy_i(self, y):
        y = y.astype('float')
        size = np.sum(y)
        return - np.sum(y / size * np.log(y / size))

    def __gini(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        return 1 - np.sum((l_c ** 2) / l_s + (r_c ** 2) / r_s,
                          axis=1) / ((l_s + r_s).flatten())

    def __entropy(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        return (- np.sum(l_c * np.log(l_c / l_s + 3e-10),
                         axis=1) - np.sum(r_c * np.log(r_c / r_s + 3e-10),
                         axis=1)) / ((l_s + r_s).flatten())

    def __misclass(self, l_c, l_s, r_c, r_s):
        return 1 - (np.max(l_c, axis=1) +
                    np.max(r_c, axis=1)) / ((l_s + r_s).flatten())

    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = np.arange(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[:int(round(np.sqrt(n_feature)))+1]

    def __get_feature_ids_log2(self, n_feature):
        feature_ids = np.arange(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[:int(round(np.log2(n_feature)))+1]

    def __get_feature_ids_N(self, n_feature):
        return np.arange(n_feature)

    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y, all_class_count):
        sorted_x, sorted_y = self.__sort_samples(x, y)
        splitted_y = sorted_y[self.min_samples_split:-self.min_samples_split]
        r_border_ids = np.where(splitted_y[:-1] != splitted_y[1:])[0]
        r_border_ids += self.min_samples_split + 1
        if len(r_border_ids) == 0:
            return float('+inf'), None
        eq_el_count = r_border_ids - np.append([self.min_samples_split],
                                               r_border_ids[:-1])
        one_hot_code = np.zeros((r_border_ids.shape[0], self.num_class))
        one_hot_code[np.arange(r_border_ids.shape[0]),
                     sorted_y[r_border_ids - 1]] = 1
        class_increments = one_hot_code * eq_el_count.reshape(-1, 1)
        box = np.bincount(sorted_y[:self.min_samples_split],
                          minlength=self.num_class)
        class_increments[0] = class_increments[0] + box
        l_class_count = np.cumsum(class_increments, axis=0)
        r_class_count = all_class_count - l_class_count
        l_sizes = r_border_ids.reshape(l_class_count.shape[0], 1)
        r_sizes = sorted_y.shape[0] - l_sizes
        gs = self.G_function(l_class_count, l_sizes, r_class_count, r_sizes)
        idx = np.argmin(gs)
        left_el_id = l_sizes[idx][0]
        return gs[idx], (sorted_x[left_el_id-1] + sorted_x[left_el_id]) / 2.0

    def __fit_node(self, x, y, node_id, depth):
        if y.size == 0:
            return
        if y.size < self.min_samples_split or \
        (self.max_depth and depth >= self.max_depth):
            class_ = np.bincount(y)
            self.tree[node_id] = [self.__class__.LEAF_TYPE,
                                  np.argmax(class_), self.G_impurity(class_)]
            return
        class_count = np.bincount(y, minlength=self.num_class)
        split_info = np.array([self.__find_threshold(x[:, ftr],
                                                     y, class_count)
                               for ftr in self.get_feature_ids(x.shape[1])])
        feature = np.argmin(split_info[:, 0])
        gs = split_info[feature, 0]
        threshold = split_info[feature, 1]
        if threshold is None:
            self.tree[node_id] = [self.__class__.LEAF_TYPE,
                                  np.argmax(np.bincount(y)), 1 - gs]
            return
        self.tree[node_id] = [self.__class__.NON_LEAF_TYPE, feature, threshold]
        l_x, r_x, l_y, r_y = self.__div_samples(x, y, feature, threshold)
        if l_x.size == 0 or r_x.size == 0:
            self.tree[node_id] = [self.__class__.LEAF_TYPE,
                                  np.argmax(np.bincount(y)), 1 - gs]
            return
        self.__fit_node(l_x, l_y, 2 * node_id + 1, depth + 1)
        self.__fit_node(r_x, r_y, 2 * node_id + 2, depth + 1)

    def fit(self, x, y):
        y = y.astype('int64')
        self.num_class = np.unique(y).size
        self.feature_importances_ = np.zeros(x.shape[1])
        self.target_size = y.size
        self.__fit_node(x, y, 0, 0)

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]

    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])

    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

In [3]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

In [4]:
wine = load_wine()
X_train, X_test, y_train, y_test = train_test_split(wine.data,
                                                    wine.target,
                                                    test_size=0.1,
                                                    stratify=wine.target)

## Проверка скорости работы на wine

In [5]:
%time clf.fit(X_train, y_train)

Wall time: 7.97 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [6]:
%time my_clf.fit(X_train, y_train)

Wall time: 69.8 ms


## Проверка качества работы на wine

In [7]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.8897028897028897

In [8]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.8897028897028897

## Подготовка данных Speed Dating Data 

In [9]:
df = pd.read_csv('Speed Dating Data.csv', encoding='ansi')
for i in df.columns:
    if type(df[i][0]) not in [np.int64, np.float64]:
        df.drop(i,axis=1, inplace=True)
for i in df.columns:
    if df[i].isnull().sum()>200:
        df.drop(i,axis=1, inplace=True)    
df=df.dropna()

In [10]:
y = np.array(df['gender'])
df[df['race'] == 6.0] = 5.0
X = np.array(df.drop(['gender', 'id'] , axis=1))
keys = np.array(df.drop(['iid', 'id', 'gender'], axis=1).keys())
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

После подготовки данных оказывается, что подходящих для изучения сэмплов о коренных американцах не остается в выборке (вероятно, из-за их малой численности и менталитета), поэтому я изменил нумерацию рас (поле 'race') на следующую: <br>
>**Black/African American**:    1 <br>
**European/Caucasian-American**:    2 <br>
**Latino/Hispanic American**:    3 <br>
**Asian/Pacific Islander/Asian-American**:    4 <br>
**Other**:    5


In [11]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

In [12]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2, criterion='entropy')
clf = DecisionTreeClassifier(min_samples_split=2, criterion='entropy')

## Проверка скорости работы на Speed Dating Data

In [13]:
%time clf.fit(X_train, y_train)

Wall time: 375 ms


DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [14]:
%time my_clf.fit(X_train, y_train)

Wall time: 3.47 s


## Проверка качества работы на Speed Dating Data

In [15]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.9624162715436141

In [16]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.954923273657289

## Задание 3

In [17]:
def feature_importance(clf, node_id):
    node = clf.tree[node_id]
    if node[0] == clf.__class__.NON_LEAF_TYPE:
        clf.feature_importances_[node[1]] += 1
        feature_importance(clf, 2 * node_id + 1)
        feature_importance(clf, 2 * node_id + 2)
    if node_id == 0:
        clf.feature_importances_ /= np.sum(clf.feature_importances_)
feature_importance(my_clf, 0)        

## Задание 4

In [19]:
print('clf', 'my_clf', sep='\t\t')
clf_keys = keys[np.argsort(clf.feature_importances_)[-10:]]
my_clf_keys = keys[np.argsort(my_clf.feature_importances_)[-10:]]
for i in range(10):
    print(clf_keys[i], end='\t')
    if len(clf_keys[i]) <= 7:
        print('\t', end='')
    print(my_clf_keys[i])

clf		my_clf
sinc3_1		imprelig
attr2_1		pf_o_sin
fun2_1		sinc1_1
movies		shar2_1
shar2_1		match
yoga		shar1_1
shar1_1		sinc2_1
sinc2_1		condtn
pf_o_sin	yoga
sinc1_1		idg


## Задание 5

In [20]:
clf = RandomForestClassifier(n_estimators=20)
param_dist = {
              "max_features": range(1, 10),
              "min_samples_split": range(2, 4),
              "criterion": ["gini", "entropy"]
            }

In [21]:
random_search = RandomizedSearchCV(clf, param_distributions=param_dist,
                                   n_iter=20, cv=5)
grid_search = GridSearchCV(clf, param_grid=param_dist, cv=5)

In [22]:
%time random_search.fit(X_train, y_train)

Wall time: 54.2 s


RandomizedSearchCV(cv=5, error_score='raise-deprecating',
          estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
          fit_params=None, iid='warn', n_iter=20, n_jobs=None,
          param_distributions={'max_features': range(1, 10), 'min_samples_split': range(2, 4), 'criterion': ['gini', 'entropy']},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring=None, verbose=0)

In [23]:
%time grid_search.fit(X_train, y_train)

Wall time: 1min 26s


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'max_features': range(1, 10), 'min_samples_split': range(2, 4), 'criterion': ['gini', 'entropy']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [24]:
f1_score(y_pred=random_search.predict(X_test), y_true=y_test, average='macro')

0.969941689134115

In [25]:
f1_score(y_pred=grid_search.predict(X_test), y_true=y_test, average='macro')

0.9711959699122729